## Cleansing Data: Book-to-Film Adaptations
This project predicts the likelihood of books becoming films and finding success on the big screen. 

We answered the question, “Which characteristics of books are strongly correlated to highly grossing, and positively rated book-to-movie adaptations?” The book characteristics that we explored, and hoped would be associated with film success, were book ratings, page counts, book genres, book audiences, and publisher type. 

In [2]:
# Import dependencies
import pandas as pd
import matplotlib as pyplot
import numpy as np
import seaborn as sns
from scipy import stats

In [3]:
# Reference csv file paths
goodreads_path = "Raw_Data/books.csv"
keywords_path = "Raw_Data/keywords.csv"
movies_path = "Raw_Data/movies_metadata.csv"
ratings_path = "Raw_Data/ratings.csv"
link_path = "Raw_Data/links.csv"

# Import csv files as DataFrames
# Book data
goodreads_df = pd.read_csv(goodreads_path, encoding="utf-8")
# Movie data
keywords_df = pd.read_csv(keywords_path, encoding="utf-8")
movies_df = pd.read_csv(movies_path, encoding="utf-8")
ratings_df = pd.read_csv(ratings_path, encoding="utf-8")
# Id linking data
link_df = pd.read_csv(link_path, encoding="utf-8")

C:\Users\laure\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# Filter df nested dict for 'based on novel' to find book-to-movie adaptations
adaptations = keywords_df[['based on novel' in row for row in keywords_df['keywords']]]

# Convert int to str to merge dfs
adaptations['id'] = adaptations['id'].astype('str')
adaptations

<ipython-input-4-90e40f5aeb14>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adaptations['id'] = adaptations['id'].astype('str')


,id,keywords
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
16,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '..."
20,8012,"[{'id': 395, 'name': 'gambling'}, {'id': 416, ..."
70,11859,"[{'id': 258, 'name': 'bomb'}, {'id': 416, 'nam..."
136,39428,"[{'id': 531, 'name': 'southern usa'}, {'id': 8..."
...,...,...
45485,32066,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
45616,339987,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name..."
45683,20650,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
45967,353491,"[{'id': 75, 'name': 'gunslinger'}, {'id': 818,..."


In [4]:
# Join dfs on 'id' field 
adaptations_merge = pd.merge(adaptations, movies_df,  how = 'inner', on = 'id')

# Visualize
adaptations_merge.head(2)

,id,keywords,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':...",False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0114885,en,Waiting to Exhale,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
1,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,NaN,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,tt0114388,en,Sense and Sensibility,...,1995-12-13,135000000.0,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Lose your heart and come to your senses.,Sense and Sensibility,False,7.2,364.0


In [5]:
# Select subset of necessary columns for analysis
adaptations_merge = adaptations_merge[['id', 'keywords', 'adult', 'budget', 'genres', 'imdb_id',
                                       'original_language', 'original_title', 'release_date', 
                                      'revenue', 'runtime', 'spoken_languages', 'status', 
                                       'vote_average', 'title']]

# Select movies that have already been released for analysis 
adaptations_merge = adaptations_merge[adaptations_merge['status'] == 'Released']

# Drop rows with NaN values
adaptations_merge = adaptations_merge.dropna(how = 'any')

# Visualize
adaptations_merge.head(2)

,id,keywords,adult,budget,genres,imdb_id,original_language,original_title,release_date,revenue,runtime,spoken_languages,status,vote_average,title
0,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':...",False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0114885,en,Waiting to Exhale,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,6.1,Waiting to Exhale
1,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",tt0114388,en,Sense and Sensibility,1995-12-13,135000000.0,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,7.2,Sense and Sensibility


There are 829 released films based on novels. 

In [6]:
# Drop null rows and convert float to int to remove decimal, then int to str to merge dfs on same datatype
link_df = link_df.dropna()
link_df['tmdbId'] = link_df['tmdbId'].astype(np.int64)
link_df['tmdbId'] = link_df['tmdbId'].astype(np.str)

# Rename tmdbId to id for merging
link_df = link_df.rename(columns = {'movieId': "movieId", 'tmdbId': 'id'})

# Merge movie and ratings data on id field
movies_merge = pd.merge(adaptations_merge, link_df,  how = 'inner', on ='id')
movies_merge.head(3)

<ipython-input-6-d413883879b0>:4: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  link_df['tmdbId'] = link_df['tmdbId'].astype(np.str)


,id,keywords,adult,budget,genres,imdb_id,original_language,original_title,release_date,revenue,runtime,spoken_languages,status,vote_average,title,movieId,imdbId
0,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':...",False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0114885,en,Waiting to Exhale,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,6.1,Waiting to Exhale,4,114885
1,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",tt0114388,en,Sense and Sensibility,1995-12-13,135000000.0,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,7.2,Sense and Sensibility,17,114388
2,8012,"[{'id': 395, 'name': 'gambling'}, {'id': 416, ...",False,30250000,"[{'id': 35, 'name': 'Comedy'}, {'id': 53, 'nam...",tt0113161,en,Get Shorty,1995-10-20,115101622.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,6.4,Get Shorty,21,113161


In [7]:
# Select subset of necessary columns for analysis
ratings_df = ratings_df[['movieId', 'rating']]

# View data types
ratings_df.info()

# View shape
ratings_df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 2 columns):
 #   Column   Dtype  
---  ------   -----  
 0   movieId  int64  
 1   rating   float64
dtypes: float64(1), int64(1)
memory usage: 397.1 MB


(26024289, 2)

There are ~26M ratings provided for 45,115 unique movies.

In [8]:
# Testing movies
test = movies_merge[movies_merge['movieId'] == 356]
test

,id,keywords,adult,budget,genres,imdb_id,original_language,original_title,release_date,revenue,runtime,spoken_languages,status,vote_average,title,movieId,imdbId
13,13,"[{'id': 422, 'name': 'vietnam veteran'}, {'id'...",False,55000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0109830,en,Forrest Gump,1994-07-06,677945399.0,142.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,8.2,Forrest Gump,356,109830


In [9]:
# How many movies reviewed in dataset
num_movies = ratings_df.value_counts('movieId')
num_movies

movieId
356       91921
318       91082
296       87901
593       84078
2571      77960
          ...  
151575        1
151581        1
113014        1
151589        1
176275        1
Length: 45115, dtype: int64

In [10]:
# Groupby id to calculate mean of user ratings
ratings_group = ratings_df.groupby('movieId')
ratings_mean = ratings_group['rating'].mean().round(2).reset_index()
ratings_mean.head()

,movieId,rating
0,1,3.89
1,2,3.24
2,3,3.18
3,4,2.88
4,5,3.08


In [11]:
# Merge ratings and movie adaptations df
final_movies_df = pd.merge(movies_merge, ratings_mean, how = 'left', on = 'movieId')
final_movies_df.head()

,id,keywords,adult,budget,genres,imdb_id,original_language,original_title,release_date,revenue,runtime,spoken_languages,status,vote_average,title,movieId,imdbId,rating
0,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':...",False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0114885,en,Waiting to Exhale,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,6.1,Waiting to Exhale,4,114885,2.88
1,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",tt0114388,en,Sense and Sensibility,1995-12-13,135000000.0,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,7.2,Sense and Sensibility,17,114388,3.95
2,8012,"[{'id': 395, 'name': 'gambling'}, {'id': 416, ...",False,30250000,"[{'id': 35, 'name': 'Comedy'}, {'id': 53, 'nam...",tt0113161,en,Get Shorty,1995-10-20,115101622.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,6.4,Get Shorty,21,113161,3.57
3,11859,"[{'id': 258, 'name': 'bomb'}, {'id': 416, 'nam...",False,50000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",tt0113010,en,Fair Game,1995-11-03,11534477.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,4.1,Fair Game,71,113010,2.35
4,39428,"[{'id': 531, 'name': 'southern usa'}, {'id': 8...",False,0,"[{'id': 18, 'name': 'Drama'}]",tt0113952,en,The Neon Bible,1995-08-23,0.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,7.2,The Neon Bible,138,113952,3.28


In [12]:
# Select subset of necessary columns for analysis
goodreads_df = goodreads_df[['title', 'authors', 'average_rating','  num_pages', 'ratings_count', 'publisher']]
# ['isbn', 'bookID', 'language_code', 'publication_date'], inplace=True)
goodreads_df.head()

,title,authors,average_rating,num_pages,ratings_count,publisher
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,652,2095690,Scholastic Inc.
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,870,2153167,Scholastic Inc.
2,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,352,6333,Scholastic
3,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,435,2339585,Scholastic Inc.
4,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,2690,41428,Scholastic


In [13]:
# Merge Goodreads data with final_movies_df
final_df = pd.merge(final_movies_df, goodreads_df, how = 'inner', on = 'title')
final_df.head(3)

,id,keywords,adult,budget,genres,imdb_id,original_language,original_title,release_date,revenue,...,vote_average,title,movieId,imdbId,rating,authors,average_rating,num_pages,ratings_count,publisher
0,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",tt0114388,en,Sense and Sensibility,1995-12-13,135000000.0,...,7.2,Sense and Sensibility,17,114388,3.95,Jane Austen/Claudia L. Johnson,4.07,269,768,W. W. Norton & Company
1,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",tt0114388,en,Sense and Sensibility,1995-12-13,135000000.0,...,7.2,Sense and Sensibility,17,114388,3.95,Jane Austen/Claire Lamont/James Kinsley,4.07,327,6853,Oxford University Press
2,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",tt0114388,en,Sense and Sensibility,1995-12-13,135000000.0,...,7.2,Sense and Sensibility,17,114388,3.95,Jane Austen/Claire Lamont/Ros Ballaster,4.07,346,528,Penguin Classics


In [14]:
# Group by title to sum book ratings
book_ratings = final_df[['title', 'ratings_count']]
groupby_book = book_ratings.groupby('title')
book_ratings_df = groupby_book.sum()
book_ratings_df.shape

(105, 1)

In [15]:
# Drop duplicates from final_df
final_df = final_df.drop_duplicates(subset = ['title'], keep = 'first')
final_df.head(1)
# Verify movie & book count
final_df.shape

(105, 23)

There are 105 matching books and movie adaptations. 

In [16]:
# Create new column for sum of ratings_count 
final_df['sum_of_ratings'] = book_ratings_df['ratings_count'].values
final_df.reset_index(inplace = True, drop=True)
final_df.head(2)

,id,keywords,adult,budget,genres,imdb_id,original_language,original_title,release_date,revenue,...,title,movieId,imdbId,rating,authors,average_rating,num_pages,ratings_count,publisher,sum_of_ratings
0,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",tt0114388,en,Sense and Sensibility,1995-12-13,135000000.0,...,Sense and Sensibility,17,114388,3.95,Jane Austen/Claudia L. Johnson,4.07,269,768,W. W. Norton & Company,1773
1,10533,"[{'id': 818, 'name': 'based on novel'}, {'id':...",False,50000000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",tt0114345,en,The Scarlet Letter,1995-10-13,10382407.0,...,The Scarlet Letter,191,114345,2.69,Nathaniel Hawthorne/Thomas E. Connolly/Nina Baym,3.4,279,609586,Penguin Books,11030


In [17]:
# Convert budget column to numeric and replace 0's with NaN to find missing values
final_df['budget'] = pd.to_numeric(final_df['budget'], errors='coerce')
final_df['budget'] = final_df['budget'].replace(0, np.nan)
final_df[final_df['budget'].isnull()].shape

(49, 24)

In [18]:
# Replace 0's with NaN to detect missing values in revenue column
final_df['revenue'] = final_df['revenue'].replace(0, np.nan)
final_df[final_df['revenue'].isnull()].shape

(47, 24)

In [19]:
# Create new column for the ratio of revenue to budget and detect missing values 
final_df['return'] = final_df['revenue'] / final_df['budget']
final_df[final_df['return'].isnull()].shape

(55, 25)

In [20]:
# Drop unneeded columns from final_df
final_df = final_df.drop(['adult','imdb_id','imdbId', 'status', 'ratings_count', 'movieId', 
                          'original_title'], axis=1)

In [21]:
# Sort values to find earliest movie release date
final_df.sort_values('release_date').head(2)

,id,keywords,budget,genres,original_language,release_date,revenue,runtime,spoken_languages,vote_average,title,rating,authors,average_rating,num_pages,publisher,sum_of_ratings,return
76,2929,"[{'id': 279, 'name': 'usa'}, {'id': 818, 'name...",NaN,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",en,1910-03-18,NaN,13.0,"[{'iso_639_1': 'xx', 'name': 'No Language'}]",6.4,Frankenstein,3.18,Mary Wollstonecraft Shelley/J. Paul Hunter,3.79,336,W. W. Norton & Company,111068,NaN
20,138,"[{'id': 272, 'name': 'transylvania'}, {'id': 8...",355000.0,"[{'id': 27, 'name': 'Horror'}]",en,1931-02-12,1012189.0,72.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",7.2,Dracula,3.69,Bram Stoker/Jan Needle/Gary Blythe,3.99,336,Walker Books Ltd,197993,2.851237


The earliest movie released was in 1910.

In [22]:
# Rename columns to clean df
final_df = final_df.rename(columns = {'keywords': 'movie keywords', 'budget': 'movie budget', 
                                      'genres': 'movie genre', 'original_language': 'movie original language', 
                                      'release_date': 'movie release date', 'revenue': 'movie revenue',
                                      'runtime': 'movie runtime', 
                                      'spoken_languages': 'movie spoken languages',
                                      'title': 'movie title', 'rating': 'movie rating', 
                                      'authors': 'book authors', 'average_rating' : 'book rating', 
                                      '  num_pages': 'book pages', 'ratings_count': 'book ratings count', 
                                      'publisher' : 'book publisher', 'sum_of_ratings': 'book ratings sum', 
                                      'return': 'movie return'})

In [23]:
# Export full dataset for analysis
final_df.to_csv('book_to_film_adaptations.csv')

In [24]:
# Drop NaN values to find book-to-film adaptations with revenue and budget data
movie_return_df = final_df.dropna(axis=0, subset=['movie return'])
movie_return_df.shape

(50, 18)

There are 50 movies with revenue and budget data

In [25]:
# Export csv of revenue and budget data for analysis
movie_return_df.to_csv('movie_return.csv')

In [26]:
final_df.head(2)

,id,movie keywords,movie budget,movie genre,movie original language,movie release date,movie revenue,movie runtime,movie spoken languages,vote_average,movie title,movie rating,book authors,book rating,book pages,book publisher,book ratings sum,movie return
0,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",16500000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",en,1995-12-13,135000000.0,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",7.2,Sense and Sensibility,3.95,Jane Austen/Claudia L. Johnson,4.07,269,W. W. Norton & Company,1773,8.181818
1,10533,"[{'id': 818, 'name': 'based on novel'}, {'id':...",50000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",en,1995-10-13,10382407.0,135.0,"[{'iso_639_1': 'en', 'name': 'English'}]",5.5,The Scarlet Letter,2.69,Nathaniel Hawthorne/Thomas E. Connolly/Nina Baym,3.4,279,Penguin Books,11030,0.207648
